In [1]:
# !pip install sdv


In [3]:
import os

# # Uninstall conflicting packages
# os.system("pip uninstall -y botocore aiobotocore")

# # Install SDV library
# os.system("pip install sdv")

# # (Optional) Reinstall aiobotocore with compatible versions
# os.system("pip install aiobotocore==2.7.0 botocore==1.31.16")

# Import necessary libraries
import numpy as np
import pandas as pd
from sdv.tabular import GaussianCopula

ModuleNotFoundError: No module named 'sdv.tabular'

In [14]:
# Load the CSV File

In [15]:
# Load the dataset
cocktails_file_path = 'cocktails.csv'
cocktails_df = pd.read_csv(cocktails_file_path)

# Replace all cells with '-' with blank
cocktails_df.replace('-', None, inplace=True)

# Assuming 'cocktails_df' is predefined with cocktail names
cocktail_ids = {name: i for i, name in enumerate(cocktails_df['Name'].unique())}

# Display the first few rows of the updated dataset
cocktails_df.head()

,Alcohol,Name,Category,Making,Base Wine,Base Wine Amount,Liquor,Liquor Amount,Juice,Juice Amount,...,Soda Amount,Others,Taste,Type of Glass,Salty,Savory,Sour,Bitter,Sweet,Spicy
0,5,Tequila Sunset,Short,Blend,Tequila,30,None,None,Lemon,30,...,None,Ice,Mild,Champagne Saucer,0,17,83,17,50,0
1,7,Chi-Chi,Long,Shake,Vodka,30,None,None,Pineapple,80,...,None,Coconut Milk,Mild,Armagnac Glasss,0,17,67,0,50,0
2,10,Sex on the Beach,Long,Build,Vodka,15,Melon_20/Raspberry_10,30,Pineapple,80,...,None,None,Mild,Highball,0,0,83,0,50,0
3,10,Horse’s Neck,Long,Build,Brandy,45,None,None,None,None,...,200,Lemon Piece,Mild,Old Fashioned,17,17,83,0,17,0
4,11,EL Diablo,Long,Build,Tequila,30,None,None,None,None,...,250,Orange Piece,Mild,Highball,17,0,50,17,50,0


In [16]:
import pandas as pd

# Load the CSV file
file_path = 'user_rating_test.csv'
user_data = pd.read_csv(file_path)

# Display the first few rows of the data
print(user_data.head())

  user_id    cocktail_name  user_rating
0  Dharti  Tequila Sunrise            1
1  Dharti  Tequila Sunrise            1
2  Dharti     Vodka Gimlet            2
3  Dharti    Black Russian            2
4  Dharti        Manhattan            4


In [17]:
user_data['cocktail_id'] = user_data['cocktail_name'].map(cocktail_ids)
user_data

,user_id,cocktail_name,user_rating,cocktail_id
0,Dharti,Tequila Sunrise,1,8
1,Dharti,Tequila Sunrise,1,8
2,Dharti,Vodka Gimlet,2,34
3,Dharti,Black Russian,2,37
4,Dharti,Manhattan,4,40
5,Dharti,Mojito,4,25
6,Dharti,Negroni,5,23
7,Dharti,Old Fashioned,5,39
8,Dharti,Bloody Mary,5,5


In [18]:
# Prepare Data Vault Components

In [19]:
# Extract hubs
users = user_data[['user_id']].drop_duplicates().reset_index(drop=True)
cocktails = user_data[['cocktail_id', 'cocktail_name']].drop_duplicates().reset_index(drop=True)

# Extract links (user-cocktail relationships with ratings)
links = user_data[['user_id', 'cocktail_id', 'user_rating']]

# Assuming user_attributes and cocktail_attributes are not in the CSV, create synthetic ones
num_users = users.shape[0]
num_cocktails = cocktails.shape[0]

user_attributes = pd.DataFrame({
    'user_id': users['user_id'],
    'age': np.random.randint(18, 70, num_users),
    'location': np.random.choice(['Location_A', 'Location_B', 'Location_C'], num_users)
})

cocktail_attributes = pd.DataFrame({
    'cocktail_id': cocktails['cocktail_id'],
    'type': np.random.choice(['Type_A', 'Type_B', 'Type_C'], num_cocktails),
    'alcohol_content': np.random.uniform(5, 40, num_cocktails)
})

# Combine all parts to create a complete dataset
complete_data = links.merge(user_attributes, on='user_id').merge(cocktail_attributes, on='cocktail_id')

# Display the first few rows of the complete dataset
print(complete_data.head())

  user_id  cocktail_id  user_rating  age    location    type  alcohol_content
0  Dharti            8            1   57  Location_C  Type_C        31.192504
1  Dharti            8            1   57  Location_C  Type_C        31.192504
2  Dharti           34            2   57  Location_C  Type_B         9.274512
3  Dharti           37            2   57  Location_C  Type_A        39.627484
4  Dharti           40            4   57  Location_C  Type_C        30.514687


In [20]:
# Generate Synthetic Data using SDV

In [21]:
from sdv.tabular import GaussianCopula

# Train a model using SDV
model = GaussianCopula()
model.fit(complete_data)

# Generate synthetic data
synthetic_data = model.sample(num_rows=10000)  # Generate 10,000 synthetic rows
print(synthetic_data.head())

# Save the synthetic data to a CSV file
synthetic_data.to_csv('synthetic_user_ratings.csv', index=False)


ModuleNotFoundError: No module named 'sdv.tabular'